# Lab2: Summarize a website in the local server

## Import libraries

In [ ]:
import os                                 # Standard library to access system environment variables
from dotenv import load_dotenv            # Loads environment variables from a .env file
from scraper import fetch_website_contents # Custom module to scrape and extract website content
from IPython.display import Markdown, display # Utilities for rich text rendering in Jupyter Notebooks
from openai import OpenAI                 # Official OpenAI client for API interactions

## Install llama3.2 in local server

In [24]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


## Initializing the Local Ollama Client

In [26]:
# Defines the endpoint address where your local Ollama server is listening for OpenAI-compatible requests
OLLAMA_BASE_URL = "http://localhost:11434/v1"

# Creates a client instance that redirects all API calls to your local server instead of the cloud
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama') # place holder for the API key

## I kept prompts the same as the first lab to observe the difference at the end

In [ ]:
system_prompt = """
You are a detail oriented assistant that analyzes the contents of a website,
and provides a summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.

"""

In [28]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## And now let's build useful messages for GPT-4.1-mini, using a function

In [29]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

## Call API

In [21]:
def summarize(url):
    """
    Fetches content from a given URL and generates a summary using a GPT model.
    """
    # Scrape or pull the text content from the specified website URL
    website = fetch_website_contents(url)
    
    # Request a summary from the OpenAI Chat Completion API
    response = ollama.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website) # Construct the prompt with the fetched content
    )
    
    # Extract and return the final text content from the model's response
    return response.choices[0].message.content

In [22]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://www.roy-lee-ai.com/about")

# Website Summary

The website appears to be the personal blog or portfolio of Roy Lee, a data analytics graduate student pursuing an M.S. CS in AI degree at Georgia Institute of Technology.

### News/Announcements

* Roy Lee is currently seeking a full-time position in AI/LLM internship.
* Available projects:

  - Designed and authored complex instruction sets and training scenarios for foundational LLMs
  - Annotated unstructured data to support Reinforcement Learning with Human Feedback (RLHF)
  - Built machine learning models to forecast next quarter profits

## Conclusion

While it is possible to run an LLM on a local server, the limitations are clear. Because my Unified RAM (or VRAM) is limited, I cannot run large-scale models. As a result, the summary quality has decreased; it is not as detailed as the results I obtained in Lab 1.